In [1]:
!pip install -U chromadb sentence-transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 13.3 MB/s eta 

In [3]:
# Imports and setup
import chromadb
from chromadb.config import Settings
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from datetime import datetime
import os

In [4]:
# Set persistent directory
persist_dir = "./chroma_medical_memory"
os.makedirs(persist_dir, exist_ok=True)

In [7]:
# Create ChromaDB client
client = chromadb.PersistentClient(path=persist_dir)
# Create embedding function
embedding_fn = SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

# Create (or get existing) collection
collection = client.get_or_create_collection(
    name="patient_symptoms",
    embedding_function=embedding_fn
)

In [8]:
# Helper functions for store the data
def store(role, message):
    timestamp = datetime.now().isoformat()
    doc_id = f"{role}-{timestamp}"
    metadata = {"role": role, "timestamp": timestamp}
    collection.add(documents=[message], metadatas=[metadata], ids=[doc_id])

In [11]:
# Query the last 5 records
def query_last_n(role, n=5):
    results = collection.get(
        where={"role": role},
        limit=n,
        include=["documents", "metadatas"]
    )
    return list(zip(results["metadatas"], results["documents"]))

# Delete the details
def clear_history():
    collection.delete(where={"role": "patient"})
    print("Agent: All patient symptom records cleared.")

In [ ]:
# Main interactive loop
print("Symptom Tracker (type 'show' to view history, 'clear' to erase, 'exit' to quit)\n")
while True:
    try:
        user_input = input("You: ").strip()
        if user_input.lower() == "exit":
            print("Session ended.")
            break
        elif user_input.lower() == "clear":
            clear_history()
            continue
        elif user_input.lower() == "show":
            history = query_last_n("patient", n=10)
            if not history:
                print("Agent: No patient history found.")
            else:
                print("Last 10 patient symptoms:")
                for meta, doc in history:
                    print(f" - [{meta['timestamp']}] {doc}")
            continue
        # Store new symptom
        store("patient", user_input)
        print("Agent: Got it. Your symptom is recorded.\n")
    except KeyboardInterrupt:
        print("\nSession interrupted.")
        break

Symptom Tracker (type 'show' to view history, 'clear' to erase, 'exit' to quit)

Last 10 patient symptoms:
 - [2025-07-30T18:33:17.022239] hi
 - [2025-07-30T18:33:25.221341] Hi 2
 - [2025-07-30T18:33:32.146250] Mey buddy
 - [2025-07-30T18:33:35.373504] jhop ata
Agent: Got it. Your symptom is recorded.

Agent: Got it. Your symptom is recorded.

Agent: Got it. Your symptom is recorded.

Last 10 patient symptoms:
 - [2025-07-30T18:33:17.022239] hi
 - [2025-07-30T18:33:25.221341] Hi 2
 - [2025-07-30T18:33:32.146250] Mey buddy
 - [2025-07-30T18:33:35.373504] jhop ata
 - [2025-07-30T18:34:12.506995] lllsdlmsdmds
 - [2025-07-30T18:34:18.240251] kahibi sangto be
 - [2025-07-30T18:34:33.147614] kaibinako sangat jaun bho
